# Jet Energy Regression with ParticleNet

In [1]:
import tensorflow as tf
import awkward as ak
import numpy as np
import pickle
import glob
import sys
import os

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Conv2D, Dense, Dropout, Layer, Multiply, Concatenate

## Data location and feature selection

In [2]:
# parquet_dir = os.path.join('/eos/cms/store/group/phys_jetmet/dholmber/jec-dnn/preprocessed/dev')

parquet_dir = '/ssd-home/hdaniel/lab/jec-dnn/data/dev'

In [3]:
epochs = 5
batch_size = 512
loss = 'mean_absolute_error'
optimizer = 'adam'
lr = 1.0e-3
shortcut = False
dropout = 0
activation = 'relu'
initializer = 'he_normal'
pooling = 'average' # average or max
batch_norm = False

K = 16

num_conv_layers = 3
num_channels = 3
first_channel = 64
channel_scale = 2 
num_dense_layers = 3
first_layer = 256
unit_scale = 0.5

shuffle_buffer = 64

train_size = 0.6
test_size = 0.2
val_size = 0.2

num_points = 100

jet_numerical = ['log_pt', 'eta', 'mass', 'phi', 'area', 'qgl_axis2', 'qgl_ptD', 'qgl_mult']
pf_numerical = ['rel_pt', 'rel_eta', 'rel_phi', 'd0', 'dz', 'd0Err', 'dzErr', 'trkChi2', 'vtxChi2', 'puppiWeight', 'puppiWeightNoLep']

In [4]:
categorical_map = {
    'jet': {
        'partonFlavour': [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 21]
    },
    'pf': {
        'charge': [-1, 0, 1],
        'lostInnerHits': [-1, 0, 1, 2],
        'pdgId': [-211, -13, -11, 1, 2, 11, 13, 22, 130, 211],
        'pvAssocQuality': [0, 1, 4, 5, 6, 7],
        'trkQuality': [0, 1, 5]
    }
}

In [6]:
jet_categorical = []
for key, categories in categorical_map['jet'].items():
    jet_categorical.extend([f'{key}_{i}' for i in range(len(categories))])
    
pf_categorical = []
for key, categories in categorical_map['pf'].items():
    pf_categorical.extend([f'{key}_{i}' for i in range(len(categories))])

In [7]:
jet_fields = jet_numerical + jet_categorical
pf_fields = pf_numerical + pf_categorical

jet_keys = [f'jet_{field}' for field in jet_fields]
pf_keys = [f'pf_{field}' for field in pf_fields]

num_jet = len(jet_keys)
num_pf = len(pf_keys)

In [8]:
dirs = glob.glob(os.path.join(parquet_dir, '*'))
num_dirs = len(dirs)
train_split = int(train_size * num_dirs)
test_split = int(test_size * num_dirs) + train_split

train_dirs = dirs[:train_split]
test_dirs = dirs[train_split:test_split]
val_dirs = dirs[test_split:]

In [9]:
train_dirs

['/ssd-home/hdaniel/lab/jec-dnn/data/dev/3',
 '/ssd-home/hdaniel/lab/jec-dnn/data/dev/4',
 '/ssd-home/hdaniel/lab/jec-dnn/data/dev/1']

In [10]:
def read_parquet(path):
    path = path.decode()

def calculate_factors(num_layers, scale):
    factors = [1]
    for i in range(num_layers - 1):
        factors.append(scale * factors[i])
    return factors

    for field in pf_fields:
        none_padded_pf = ak.pad_none(pf[field], target=num_points, clip=True, axis=1)
        zero_padded_pf = ak.to_numpy(none_padded_pf).filled(0)
        data.append(zero_padded_pf.astype(np.float32))
    
    return data

In [11]:
def read_parquet_wrapper(path):
    inp = [path]
    Tout = [tf.int32] + [tf.float32] + [tf.float32] * num_jet + [tf.float32] * num_pf
    
    cols = tf.numpy_function(read_parquet, inp=inp, Tout=Tout)
    
    keys = ['row_lengths'] + ['target'] + jet_keys + pf_keys
    data = {key: value for key, value in zip(keys, cols)}
    
    data['target'].set_shape((None,))
    
    row_lengths = data.pop('row_lengths')
    row_lengths.set_shape((None,))
    
    for key in jet_keys:
        # Shape from <unknown> to (None,)
        data[key].set_shape((None,))
        # Shape from (None,) to (None, 1)
        data[key] = tf.expand_dims(data[key], axis=1)
    
    for key in pf_keys:
        # Shape from <unknown> to (None, P)
        data[key].set_shape((None, num_points))
        # Shape from (None, P) to (None, P, 1)
        data[key] = tf.expand_dims(data[key], axis=2)
    
    return data

In [12]:
def parse_record(example_proto):
    return tf.io.parse_single_example(example_proto, features=all_features)

def select_features(batch):
    jet_data = tf.stack([batch[key] for key in jet_keys], axis=1)
    pf_data = tf.stack([batch[key].values for key in pf_keys], axis=1)
    pf_data = tf.RaggedTensor.from_row_lengths(pf_data, row_lengths=batch['row_lengths']).to_tensor(shape=(None, num_points, num_pf))
    
    mask = tf.cast(tf.math.not_equal(pf_data[:,:,0:1], 0), dtype=tf.float32) # 1 if valid
    coord_shift = tf.multiply(1e6, tf.cast(tf.math.equal(mask, 0), dtype=tf.float32))
    points = tf.concat([pf_data[:,:,1:2], pf_data[:,:,2:3]], axis=2)
    
    inputs = (pf_data, jet_data, data['points'], data['coord_shift'], data['mask'])
    return inputs, data['target']

In [13]:
def create_dataset(paths):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(read_parquet_wrapper, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.map(prepare_pf_inputs, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.unbatch().batch(batch_size)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [14]:
train_ds = create_dataset(train_files).shuffle(shuffle_buffer)
val_ds = create_dataset(val_files)
test_ds = create_dataset(test_files)

In [15]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Conv2D, Dense, Dropout, Layer, Multiply, Concatenate

In [16]:
class Mean(Layer):
    def __init__(self, axis, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.math.reduce_mean(inputs, axis=self.axis)


class Max(Layer):
    def __init__(self, axis, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.math.reduce_max(inputs, axis=self.axis)


class Expand(Layer):
    def __init__(self, axis, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.expand_dims(inputs, axis=self.axis)


class Squeeze(Layer):
    def __init__(self, axis, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.squeeze(inputs, axis=self.axis)


In [17]:
def get_particle_net():
    """
    ParticleNet: Jet Tagging via Particle Clouds
    arxiv.org/abs/1902.08570
    
    Parameters
    ----------
    input_shapes : dict
        The shapes of each input (`points`, `features`, `mask`).
    """

    features = Input(name='features', shape=(num_points, num_pf))
    globals = Input(name='globals', shape=(num_jet,))
    points = Input(name='points', shape=(num_points, 2))
    coord_shift = Input(name='coord_shift', shape=(num_points, 1))
    mask = Input(name='mask', shape=(num_points, 1))

    outputs = particle_net_base(points, features, mask, coord_shift, globals)

    model = Model(inputs=[features, globals, points, coord_shift, mask], outputs=outputs)

    model.summary()

    return model


def particle_net_base(points, features, mask, coord_shift, globals):
    """
    points : (N, P, C_coord)
    features:  (N, P, C_features), optional
    mask: (N, P, 1), optional
    """

    fts = features
    for layer_idx, sub_channels in enumerate(channels, start=1):
        pts = Add(name=f'add_{layer_idx}')([coord_shift, points]) if layer_idx == 1 else Add(name=f'add_{layer_idx}')([coord_shift, fts])
        fts = edge_conv(
            pts, fts, num_points, sub_channels, name=f'edge_conv_{layer_idx}'
        )

    fts = Multiply()([fts, mask])

    pool = Mean(axis=1)(fts) # (N, C)

    x = Concatenate(name='head')([pool, globals])

    for layer_idx, n in enumerate(units):
        x = Dense(n)(x)
        x = Activation(activation)(x)
        if dropout:
            x = Dropout(dropout)(x)
    out = Dense(1, name='out')(x)
    return out # (N, num_classes)


def edge_conv(points, features, num_points, sub_channels, name):
    """EdgeConv
    Args:
        K: int, number of neighbors
        in_channels: # of input channels
        channels: tuple of output channels
        pooling: pooling method ('max' or 'average')
    Inputs:
        points: (N, P, C_p)
        features: (N, P, C_0)
    Returns:
        transformed points: (N, P, C_out), C_out = channels[-1]
    """

    fts = features
    knn_fts = KNearestNeighbors(num_points, K, name=f'{name}_knn')([points, fts])

    x = knn_fts
    for idx, channel in enumerate(sub_channels, start=1):
        x = Conv2D(
            channel, kernel_size=(1, 1), strides=1, data_format='channels_last',
            use_bias=False if batch_norm else True, kernel_initializer=initializer, name=f'{name}_conv_{idx}'
        )(x)
        if batch_norm:
            x = BatchNormalization(name=f'{name}_batchnorm_{idx}')(x)
        if activation:
            x = Activation(activation, name=f'{name}_activation_{idx}')(x)

    if pooling == 'max':
        fts = Max(axis=2, name=f'{name}_max')(x) # (N, P, C')
    else:
        fts = Mean(axis=2, name=f'{name}_mean')(x) # (N, P, C')

    if shortcut:
        sc = Expand(axis=2, name=f'{name}_shortcut_expand')(features)
        sc = Conv2D(
            sub_channels[-1], kernel_size=(1, 1), strides=1, data_format='channels_last',
            use_bias=False if batch_norm else True, kernel_initializer=initializer, name=f'{name}_shortcut_conv'
        )(sc)
        if batch_norm:
            sc = BatchNormalization(name=f'{name}_shortcut_batchnorm')(sc)
        sc = Squeeze(axis=2, name=f'{name}_shortcut_squeeze')(sc)

        x = Add(name=f'{name}_add')([sc, fts])
    else:
        x = fts

    return Activation(activation, name=f'{name}_activation')(x) # (N, P, C')


class KNearestNeighbors(Layer):
    def __init__(self, num_points, k, **kwargs):
        super().__init__(**kwargs)
        self.num_points = num_points
        self.k = k

    def call(self, inputs):
        points, features = inputs
        # distance
        D = batch_distance_matrix_general(points, points) # (N, P, P)
        _, top_k_indices = tf.math.top_k(-D, k=self.k + 1) # (N, P, K+1)
        top_k_indices = top_k_indices[:, :, 1:] # (N, P, K)

        queries_shape = tf.shape(features)
        batch_size = queries_shape[0]
        batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, self.num_points, self.k, 1))
        indices = tf.concat([batch_indices, tf.expand_dims(top_k_indices, axis=3)], axis=3) # (N, P, K, 2)
        
        knn_fts =  tf.gather_nd(features, indices) # (N, P, K, C)
        knn_fts_center = tf.tile(tf.expand_dims(features, axis=2), (1, 1, self.k, 1)) # (N, P, K, C)

        return tf.concat([knn_fts_center, tf.subtract(knn_fts, knn_fts_center)], axis=-1) # (N, P, K, 2*C)


# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)
def batch_distance_matrix_general(A, B):
    r_A = tf.math.reduce_sum(A * A, axis=2, keepdims=True)
    r_B = tf.math.reduce_sum(B * B, axis=2, keepdims=True)
    m = tf.linalg.matmul(A, tf.transpose(B, perm=(0, 2, 1)))
    D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 2, 1))
    return D

In [18]:
dnn = get_particle_net()
dnn.compile(optimizer=optimizer, loss=loss)
dnn.optimizer.lr.assign(lr)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
coord_shift (InputLayer)        [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
points (InputLayer)             [(None, 100, 2)]     0                                            
__________________________________________________________________________________________________
add_1 (Add)                     (None, 100, 2)       0           coord_shift[0][0]                
                                                                 points[0][0]                     
__________________________________________________________________________________________________
features (InputLayer)           [(None, 100, 37)]    0                                        

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.001>

In [19]:
# tf.keras.utils.plot_model(dnn, dpi=100, show_shapes=True, expand_nested=True)

In [20]:
def get_callbacks():
    # Reduce learning rate when nearing convergence
    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1.0e-8,
        mode='auto', min_delta=1.0e-4, cooldown=0, verbose=1
    )
    # Stop early if the network stops improving
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=1.0e-4, 
        patience=7, mode='auto', baseline=None, 
        restore_best_weights=True, verbose=1
    )

    return [reduce_lr_on_plateau, early_stopping]

In [21]:
fit = dnn.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=get_callbacks())

Epoch 1/5
2/2 [==============================] - 4s 922ms/step - loss: 2.0169 - val_loss: 1.1221
Epoch 2/5
2/2 [==============================] - 3s 553ms/step - loss: 1.1859 - val_loss: 0.7635
Epoch 3/5
2/2 [==============================] - 3s 552ms/step - loss: 0.9834 - val_loss: 0.3966
Epoch 4/5
2/2 [==============================] - 3s 3s/step - loss: 0.4978 - val_loss: 0.5362
Epoch 5/5
2/2 [==============================] - 3s 3s/step - loss: 0.4616 - val_loss: 0.5181


In [11]:
result = dnn.evaluate(test_ds)

      4/Unknown - 1s 164ms/step - loss: 0.1318

In [12]:
print(result)

0.13182656280696392
